In [3]:
%config Completer.use_jedi = False

import pandas as pd
import os
import subprocess
from video import Video
import random
from extractors import ExtractorFactory
from clusters import ClusterFactory
from mxnet import nd, gpu
from s3 import S3
import io
import cv2
import finch
from pathlib import Path
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np
import math
from pprint import pprint
from accuracy import Accuracy
from finch import FINCH
from npy import Npy

/home/guilherme/miniconda3/envs/as_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.decomposition.pca module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/guilherme/miniconda3/envs/as_env/lib/python3.7/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.7.0` and `torch==1.6.0` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '
/home/guilherme/action_eval/finch.py:13: UserWarning: pyflann not installed: No module named 'pyflann'
  warnings.warn('pyflann not installed: {}'.format(e))


# Getting Breakfast's video paths

### getting ground truth paths

In [4]:
gt_paths = {}
for path, subdirs, files in os.walk('segmentation_coarse'):
    split_path = path.split("/")
    if len(split_path) == 2:
        print(split_path)
        gt_paths[split_path[-1]] = []
        for name in files:
            if name.split(".")[-1] == 'txt':
                gt_paths[split_path[-1]].append(os.path.join(path, name))

sum([len(paths) for _, paths in gt_paths.items()])

['segmentation_coarse', 'scrambledegg']
['segmentation_coarse', 'milk']
['segmentation_coarse', 'coffee']
['segmentation_coarse', 'salat']
['segmentation_coarse', 'cereals']
['segmentation_coarse', 'juice']
['segmentation_coarse', 'sandwich']
['segmentation_coarse', 'friedegg']
['segmentation_coarse', 'pancake']
['segmentation_coarse', 'tea']


1712

### Function that transforms a ground truth file path to a video file path

In [5]:
from pathlib import Path

def gt_to_vd_path(gt_path):
    vd_root_folder = '../BreakfastII_15fps_qvga_sync'
    splitted_path = gt_path.split("/")[-1].split("_")
    pfolder = splitted_path[0]
    splitted_path[-1] = splitted_path[-1].split(".")[0]

    if 'stereo' in gt_path:
        recfolder = splitted_path[1][:-2]
        filename = "_".join([splitted_path[0], splitted_path[-1], 'ch1'])
        vd_path = "/".join([vd_root_folder, pfolder, recfolder, filename + '.avi'])
        if Path(vd_path).exists():
            return vd_path
        else:
            filename = "_".join([splitted_path[0], splitted_path[-1], 'ch0'])
            vd_path = "/".join([vd_root_folder, pfolder, recfolder, filename + '.avi'])
            return vd_path
    else:
        recfolder = splitted_path[1]
        filename = "_".join([splitted_path[0], splitted_path[-1]])
    
    return "/".join([vd_root_folder, pfolder, recfolder, filename + '.avi'])

## Getting INRIA's video paths

In [7]:
def get_video_paths(root_folder, video_ext):
    paths = []
    for path, subdirs, files in os.walk(root_folder):
        for name in files:
            if name.split(".")[-1] == video_ext:
                paths.append(os.path.join(path, name))
    return paths

paths = get_video_paths(root_folder='data_new', video_ext='avi')
len(paths)

149

## Extracting SlowFast Features

In [ ]:
clip_len = 32
extractor = ExtractorFactory.get(ExtractorFactory.SLOWFAST.value)(clip_len=clip_len)

### for Breakfast dataset

In [ ]:
for key, gts in gt_paths.items():
    print(key)
    for gt_path in gts:
        vd_path = gt_to_vd_path(gt_path)
        video = Video(vd_path, "_".join([ExtractorFactory.SLOWFAST.value, str(clip_len)]))
        print(gt_path)
        extractor.extract(video)

### for INRIA

In [ ]:
for path in paths:
    video = Video(path, "_".join([ExtractorFactory.SLOWFAST.value, str(clip_len)]))
    if video.features.has_features:
        continue
    print(path)
    extractor.extract(video)

## Extracting I3D features

### for INRIA

In [ ]:
# create this file
with open('./video_paths.txt', 'w') as paths_file:
    for path in paths:
        paths_file.write("../action_eval/"+path+"\n")

# run this command in terminal to extract the features inside the video_features folder
# please, remember to substitute
# !python main.py --feature_type i3d --device_ids 0 --stack_size 10 --step_size 10 --output_path ../action_eval/inria_features --on_extraction save_numpy --file_with_video_paths ../action_eval/video_paths.txt
# !python main.py --feature_type i3d --device_ids 0 --stack_size 16 --step_size 16 --output_path ../action_eval/inria_features --on_extraction save_numpy --file_with_video_paths ../action_eval/video_paths.txt
# !python main.py --feature_type i3d --device_ids 0 --stack_size 24 --step_size 24 --output_path ../action_eval/inria_features --on_extraction save_numpy --file_with_video_paths ../action_eval/video_paths.txt
# !python main.py --feature_type i3d --device_ids 0 --stack_size 32 --step_size 32 --output_path ../action_eval/inria_features --on_extraction save_numpy --file_with_video_paths ../action_eval/video_paths.txt
# !python main.py --feature_type i3d --device_ids 0 --stack_size 40 --step_size 40 --output_path ../action_eval/inria_features --on_extraction save_numpy --file_with_video_paths ../action_eval/video_paths.txt
# !python main.py --feature_type i3d --device_ids 0 --stack_size 48 --step_size 48 --output_path ../action_eval/inria_features --on_extraction save_numpy --file_with_video_paths ../action_eval/video_paths.txt
# !python main.py --feature_type i3d --device_ids 0 --stack_size 64 --step_size 64 --output_path ../action_eval/inria_features --on_extraction save_numpy --file_with_video_paths ../action_eval/video_paths.txt

### for Breakfast dataset

Copy the `extract_features.py` file to the video_features folder and run the command below.
```
python extract_features.py
```
Don't forget to run this script for each segment size mentioned in the paper, just substitute the proper var inside the script to do this,